# Zero-Shot Learning for Record Linkage

## Load database

In [1]:
%load_ext autoreload
%autoreload 2

## Instantiate collection 

In [6]:
import re
import transformers  # old version '4.25.1'
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
from tools.weaving_tools import SMGCollection,NMSCollection

In [7]:
collection = NMSCollection(img_folder=Path('nms_imgs'))
collection.load_from_csv('data/NMS.csv')
collection.filter_records()

# Mapping Images to Taxonomies

In [8]:

checkpoint = 'openai/clip-vit-base-patch32'
model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

In [13]:
matches = [re.search(r'[A-Z\-\s]+',str(t)) for t in collection.df.taxonomy.unique()]
matches = set([m.group().strip() for m in matches if m])
candidate_labels = list({i for i in matches if len(i) > 1})
candidate_labels.append('OTHER')
candidate_labels

['OFFICE EQUIPMENT',
 'CIVIL ENGINEERING',
 'PHOTOGRAPHY',
 'DOMESTIC',
 'MINERAL OILS',
 'TRANSPORT',
 'MEDICINE',
 'ROAD TRANSPORT',
 'NON-FERROUS METALS',
 'GAS',
 'INDUSTRIES',
 'AUDIO',
 'PNEUMATICS',
 'MINING',
 'TELECOMMUNICATIONS',
 'SHIPPING',
 'RAIL TRANSPORT',
 'COMPUTING',
 'PHYSICS',
 'SPORT',
 'MECHANICAL ENGINEERING',
 'CHEMISTRY',
 'HEAT ENGINES',
 'AERONAUTICS',
 'EXPLOSIVES',
 'DOMESTIC EQUIPMENT',
 'ELECTRICAL AND ELECTRONIC ENGINEERING',
 'FERROUS METALS',
 'CARTOGRAPHY',
 'CIVIL AND MECHANICAL ENGINEERING',
 'BT C',
 'HYDRAULICS',
 'OTHER']

In [ ]:
inputs = processor(images=image, text=candidate_labels, return_tensors="pt", padding=True)

In [ ]:
#inputs

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)
    
logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

In [ ]:
result = [
    {"score": score, "label": candidate_label}
        for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

In [ ]:
result

In [ ]:
import pandas as pd
pd.Series(scores, index=candidate_labels).plot(kind='bar')

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

result = [
    {"score": score, "label": candidate_label}
        for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

result

# Nomic Experiment

In [ ]:
from nomic import atlas
import numpy as np

In [ ]:
%%bash
nomic login pyKINnSOXSivweuAXxprY7j8NtVFI88WC7AeE3BnLAmQZ

In [ ]:
nms_clip_embeddigns = np.array(nms.dataset['clip_image_embedding'])
base_url = 'https://www.nms.ac.uk/search.axd?command=getcontent&server=Detail&value='
nms.df['url'] = nms.df.img_loc.apply(lambda x: base_url + x)
nms_data = [{'category': str(row.taxonomy), 'collection':'nms','id': str(i), 'url': row.url}
            for i, row in nms.df.iterrows()]

In [ ]:
smg_clip_embeddigns = np.array(smg.dataset['clip_image_embedding'])
base_url = 'https://coimages.sciencemuseumgroup.org.uk/images/'
smg.df['url'] = smg.df.img_loc.apply(lambda x: base_url + x)
smg_data = [{'category': str(row.taxonomy), 'collection':'smg','id': str(i), 'url': row.url}
            for i, row in smg.df.iterrows()]

In [ ]:
embeddings = np.concatenate([nms_clip_embeddigns,smg_clip_embeddigns], axis=0)

In [ ]:
data = nms_data + smg_data

In [ ]:
len(data), len(embeddings)

In [ ]:
project = atlas.map_embeddings(embeddings=embeddings,
                                data=data,
                                id_field='id',
                                name='CE Map',
                                colorable_fields=['category','collection'],
                                reset_project_if_exists=True)

In [ ]:
nms.df['tax_simpl'] = nms.df.taxonomy.apply(lambda x: re.search(r'[A-Z\-\s]+', str(x)))
nms.df['tax_simpl'] = nms.df['tax_simpl'].apply(lambda x: x.group() if x else 'OTHER')

In [ ]:
clip_embeddigns.shape, len(data)

In [ ]:
print(project)

In [ ]:
dir(nms)

In [ ]:
nms.load_clip_model()

In [ ]:
from transformers import pipeline

captioner = pipeline("image-to-text",model="google/pix2struct-textcaps-base")

In [ ]:

idx = 302
captioner(nms.df.iloc[idx].img_path)

In [ ]:
Image.open(nms.df.iloc[idx].img_path)

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset
from PIL import Image
import os
df = nms.df.copy()

# Define a function to load images from file paths
def load_image(image_path):
    return Image.open(image_path)

# Apply the image loading function to the DataFrame
df["image"] = df["img_path"]#.apply(load_image)
df.rename({'description':'text'}, axis=1, inplace=True)
# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df[['text','image']])




In [ ]:
dataset

# Network

In [ ]:
ids2ints = {}

for i,r_id in enumerate(list(nms.df.record_id)):
    ids2ints[r_id] = i
    
for i,r_id in enumerate(list(smg.df.record_id)):
    ids2ints[r_id] = nms.df.shape[0] + i   

In [ ]:
nodes = list(ids2ints.values())

In [ ]:
def result_to_edgelist(results,e1):
    df = pd.DataFrame({k: v[0] for k,v in results.items() if k !='embeddings'} )
    df = pd.concat([df,df.metadatas.apply(pd.Series)], axis=1)
    df['distances'] = (1 / (1+ df.distances)) * 100
    return [(e1,
            ids2ints[row.record_id], 
            row.distances
                        ) for i, row in df.iterrows() 
                if e1 != ids2ints[row.record_id]]

In [ ]:
from tqdm.notebook import tqdm
nodes = []
names = []
edges = []

for i in tqdm(range(1000)): #len(nms.dataset)
    
    
    results_smg = collection.query(
            query_embeddings=[list(np.array(nms.dataset[i]['clip_image_embedding']).astype(float))],
            n_results=3,
            where={"collection": "smg"},
                )
    e1 = ids2ints[nms.dataset[i]['record_id']]
    
    edges.extend(result_to_edgelist(results_smg,e1))
    
    
    results_nms = collection.query(
            query_embeddings=[list(np.array(nms.dataset[i]['clip_image_embedding']).astype(float))],
            n_results=3,
            where={"collection": "nms"},
                )
    
    edges.extend(result_to_edgelist(results_nms,e1))

In [ ]:
nodes = []
for i in edges:
    nodes.extend(i[:2])
nodes = list(set(nodes))

In [ ]:
ints2ids = {v : k for k,v in ids2ints.items()}

In [ ]:
label = [str(ints2ids[n]) for n in nodes] 

In [ ]:
from pyvis.network import Network
#net = Network(notebook=True)
net = Network(height="750px", 
              width="100%", 
              bgcolor="#222222", 
              font_color="white", 
              cdn_resources='remote', 
              #filter_menu=True, 
              notebook=True)

net.add_nodes(
    nodes,
    label=label,
    color=["#00bfff" if i < nms.df.shape[0] else "#ffc0cb" for i in nodes],
)

In [ ]:
net.add_edges(edges)

In [ ]:
#net.show("network.html")

In [ ]:
#net.show_buttons(filter_=['physics'])

In [ ]:
net = draw_network(
    nodes, df, minium_weight=0, repulsion=100, spring_length=500, buttons=["physics"]
)
net.show("match_with_buttons.html")

In [ ]:
net.show_buttons()

In [ ]:
# Image to Text

In [3]:
!pip install bitsandbytes sentencepiece accelerate transformers

In [1]:
checkpoint = "HuggingFaceM4/idefics-9b"

In [2]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

processor = AutoProcessor.from_pretrained(checkpoint)

model = IdeficsForVisionText2Text.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [6]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
